# **5. Deploying Machine Learning Models**

# **5.1 Intro / Session overview** (video 1)

## **Notes**
____
In This session, we talked about the earlier model we made in chapter 3 for churn prediction.

This chapter contains the deployment of the model. If we want to use the model to predict new values without running the code, There's a way to do this. The way to use the model on different machines without running the code is to deploy the model on a server (run the code and make the model). After deploying the code in a machine used as server we can make some endpoints (using api's) to connect from another machine to the server and predict values.

To deploy the model in a server there are some steps:

* After training the model save it, to use it for making predictions in future (session 02-pickle).
* Make the API endpoints in order to request predictions. (session 03-flask-intro and 04-flask-deployment)
* Some other server deployment options (sessions 5 to 9)


In the previous session we trained a model for predicting churn and evaluated it. Now let's deploy it

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [ ]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv'


In [ ]:
!wget $data -O data-week-3.csv 

--2022-10-04 21:09:34--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 977501 (955K) [text/plain]
Saving to: ‘data-week-3.csv’

data-week-3.csv     100%[===================>] 954.59K  --.-KB/s    in 0.1s    

2022-10-04 21:09:36 (8.45 MB/s) - ‘data-week-3.csv’ saved [977501/977501]



In [ ]:
df = pd.read_csv('data-week-3.csv')

df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')
df.totalcharges = df.totalcharges.fillna(0)

df.churn = (df.churn == 'yes').astype(int)

In [ ]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [ ]:
numerical = ['tenure', 'monthlycharges', 'totalcharges']

categorical = [
    'gender',
    'seniorcitizen',
    'partner',
    'dependents',
    'phoneservice',
    'multiplelines',
    'internetservice',
    'onlinesecurity',
    'onlinebackup',
    'deviceprotection',
    'techsupport',
    'streamingtv',
    'streamingmovies',
    'contract',
    'paperlessbilling',
    'paymentmethod',
]

In [ ]:
def train(df_train, y_train, C=1.0):
    dicts = df_train[categorical + numerical].to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)

    model = LogisticRegression(C=C, max_iter=1000)
    model.fit(X_train, y_train)
    
    return dv, model

In [ ]:
def predict(df, dv, model):
    dicts = df[categorical + numerical].to_dict(orient='records')

    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:, 1]

    return y_pred

In [ ]:
C = 1.0
n_splits = 5

In [ ]:
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

scores = []

for train_idx, val_idx in kfold.split(df_full_train):
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]

    y_train = df_train.churn.values
    y_val = df_val.churn.values

    dv, model = train(df_train, y_train, C=C)
    y_pred = predict(df_val, dv, model)

    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)

print('C=%s %.3f +- %.3f' % (C, np.mean(scores), np.std(scores)))

C=1.0 0.840 +- 0.008


In [ ]:
scores

[0.8419943324096679,
 0.8455854357038802,
 0.8311739915713425,
 0.8301684306452645,
 0.851750023532365]

In [ ]:
dv, model = train(df_full_train, df_full_train.churn.values, C=1.0)
y_pred = predict(df_test, dv, model)

auc = roc_auc_score(y_test, y_pred)
auc

NameError: ignored

In [ ]:
dv, model = train(df_full_train, df_full_train.churn.values, C=1.0)
y_pred = predict(df_test, dv, model)

y_test = df_test.churn.values
auc = roc_auc_score(y_test, y_pred)
auc

0.8572386167896259

# **5.2 Saving and loading the model**(video 2)

In this session we'll cover the idea "How to use the model in future without training and evaluating the code"

  * To save the model we made before there is an option using the pickle library:
    * First install the library with the command pip install pickle-mixin if you don't have it.
    * After training the model and being the model ready for prediction process uses this code to save the model for later.


   

## **Save the model**

* For saving the model we will use pickle which is a built-in library for saving python objects.

In [ ]:
import pickle

* Let's import it, import pickle, take our model and write it to your file, we need to first create a file where we will write it, let's call it **"output_file = f'model_C={C}.bin"**

In [ ]:
output_file = f'model_C={C}.bin'
output_file 

'model_C=1.0.bin'

* Let's create a special variable for storing the name of the output file, model underscores C, we want to know what is  the C, we use for training this model and put it on file name then bin.

In [ ]:
import pickle
with open('model.bin', 'wb') as f_out:
   pickle.dump((dv, model), f_out)
f_out.close() ## After opening any file it's nessecery to close it

  * In the code above we'll making a binary file named model.bin and writing the dict_vectorizer for one hot encoding and model as array in it. (We will save it as binary in case it wouldn't be readable by humans)
  * To be able to use the model in future without running the code, We need to open the binary file we saved before.


In [ ]:
f_out = open(output_file, 'wb') 
pickle.dump((dv, model), f_out)
f_out.close()## After opening any file it's nessecery to close it

In [ ]:
!ls -lh *.bin

-rw-r--r-- 1 root root 2.8K Oct  4 18:56 'model_C=1.0.bin'


In [ ]:
with open(output_file, 'wb') as f_out: 
    pickle.dump((dv, model), f_out)
    # do stuff

# other stuf

notes:

* Take this output file and we want to create a file with, write open function, opens a file and here we going to specify what we want to do with this file, the file to be binary.
* Use pickle to save our model, we use the dump function for that, write model and write the file **f_out** and then the last thing, we need to close the file and here I saved on the model but we have actually, two things here we have the dict vectorizer as well, look our predict function we need to have both not just the model because with just the model will not be able to translate a user or a customer into a feature matrix.
* We need to have both, write a tuple, we will write to this model underscore C file we will write two things we will write the dictionary vectorizer and write the model.
* Open the file and save the model there and we close it calling the close method s pretty important because it, we don't do this we cannot be sure if this file has the content and other service can use it.
* It's very easy to accidentally forget to close the file that's why I prefer to use with a statement that makes sure that the file is closed all the time, it automatically closes the file.
* We can do it with open output file as f out which is equivalent and put this line **"pickle.dump(dv,mode),f_out)"** then do this thing inside the statement, do stuff.
* Once we leave this with block do other stuff so everything, do here inside the statement the file is still open once we go outside of the "with" statement once, then the file is authomatically lost this is a nice way and easy way to make sure that no matter what you do the file is close to the end and it's also a bit shoter just two lines versus three and this is how we save the models.

* Let me start the kernel so we're pretending we're a different process
* just click the kernel to restart kernel [1]

Load the model



In [ ]:
import pickle

* Import pickle and you see this is now cell number one [1] because it **started the jupyter kernel** or in **google collab we call it the runtime**.
* From scratch it doesn't have to see if we write model here it says, because we restart the kernel and it doesn't have access to the variable we used here, we are starting from a clean slate.


In [ ]:
input_file = 'model_C=1.0.bin'

In [ ]:
with open(input_file, 'rb') as f_in: 
    dv, model = pickle.load(f_in)

* We want to load the model and its' pretty similar to saving the model except here, let's call if **f_in** file input and here I call the f_out which means file output, it's a file that I created for writing and it will be a file for reading and like we need to replace the "w" to "r so we read a file and this is very important if you don't if  you forget accidentally to change it and it leaves us "w" here is right there it will overwrite the file so it will just create a new file with zero bytes we don't wan to have that, we want to open this file for reading adn instead of dump we use load, load reads from the file, we have file in and it returns the thing we saved, this is what we saved and this is what we load.


In [ ]:
# with open('mode.bin', 'rb') as f_in:  ## Note that never open a binary file you do not trust!
    dict_vectorizer, model = pickle.load(f_in)
f_in.close()

* With unpacking the model and the dict_vectorizer, We're able to again predict new input values without training a new model by re-running the code.


In [ ]:
dv,model

(DictVectorizer(sparse=False), LogisticRegression(max_iter=1000))

* Create a variable I'll call it an input file, now we have our dict vectorizer and we have our logistic regression this is what we trained and saved previously and you can also notice that there are no imports, so I don't import sklearn here out, we have to scikit-learn installed on our computer so without that this is will not work, when it will try to load the pickle file it will complain "saving that" I don't know what this is I cannot create these classes I cannot create this dictionary vectorizer and logistic regression because scikit-learn is not installed on your computer.
* Pretending this person who we want to scores who want to send to our churn service to understand which the probability, the probability of joining


In [ ]:
customer = {
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'yes',
    'dependents': 'no',
    'phoneservice': 'no',
    'multiplelines': 'no_phone_service',
    'internetservice': 'dsl',
    'onlinesecurity': 'no',
    'onlinebackup': 'yes',
    'deviceprotection': 'no',
    'techsupport': 'no',
    'streamingtv': 'no',
    'streamingmovies': 'no',
    'contract': 'month-to-month',
    'paperlessbilling': 'yes',
    'paymentmethod': 'electronic_check',
    'tenure': 1,
    'monthlycharges': 29.85,
    'totalcharges': 29.85
}

In [ ]:
dv.transform([customer])

array([[ 1.  ,  0.  ,  0.  ,  1.  ,  0.  ,  1.  ,  0.  ,  0.  ,  1.  ,
         0.  ,  1.  ,  0.  ,  0.  , 29.85,  0.  ,  1.  ,  0.  ,  0.  ,
         0.  ,  1.  ,  1.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ,  1.  ,
         0.  ,  0.  ,  1.  ,  0.  ,  1.  ,  0.  ,  0.  ,  1.  ,  0.  ,
         0.  ,  1.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  1.  , 29.85]])

* We use our dictionary vectorizer transform method and then put the customer and the dictionary vectorizer expects a list of dictionaries that's why we create a list with just one customer and this is the output, let me save this output into **X** but,

In [ ]:
X = dv.transform([customer])

* Next well call model predict proper for this **X**, so we have a two-dimensional array as usual and we're interested in this thing here, let's write it like row number zero the column number one and we're getting the probability that this particular customer is going to churn.

In [ ]:
model.predict_proba(X)

array([[0.36364158, 0.63635842]])

In [ ]:
model.predict_proba(X)[0,1] # if tenure is 1

0.6363584152704198

In [ ]:
# if the tenure is 20
model.predict_proba(X)

array([[0.36364158, 0.63635842]])

In [ ]:
model.predict_proba(X)[0,1]# if tenure is 20 the  probability is 0.33

0.3313627488178145

In [ ]:
y_pred = model.predict_proba(X)[0, 1]

In [ ]:
print('input:', customer)
print('output:', y_pred)

input: {'gender': 'female', 'seniorcitizen': 0, 'partner': 'yes', 'dependents': 'no', 'phoneservice': 'no', 'multiplelines': 'no_phone_service', 'internetservice': 'dsl', 'onlinesecurity': 'no', 'onlinebackup': 'yes', 'deviceprotection': 'no', 'techsupport': 'no', 'streamingtv': 'no', 'streamingmovies': 'no', 'contract': 'month-to-month', 'paperlessbilling': 'yes', 'paymentmethod': 'electronic_check', 'tenure': 1, 'monthlycharges': 29.85, 'totalcharges': 29.85}
output: 0.6363584152704198


* Let's create such a script, a script that trains a model for that what we can do is we can just go to our jupyter and then download this notebook as a python.py file click on that, and it saves the file.

* Open it in Visual Studio Code.

Making requests

In [ ]:
import requests

In [ ]:
url = 'http://localhost:9696/predict'

In [ ]:
customer = {
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'yes',
    'dependents': 'no',
    'phoneservice': 'no',
    'multiplelines': 'no_phone_service',
    'internetservice': 'dsl',
    'onlinesecurity': 'no',
    'onlinebackup': 'yes',
    'deviceprotection': 'no',
    'techsupport': 'no',
    'streamingtv': 'no',
    'streamingmovies': 'no',
    'contract': 'two_year',
    'paperlessbilling': 'yes',
    'paymentmethod': 'electronic_check',
    'tenure': 1,
    'monthlycharges': 29.85,
    'totalcharges': 29.85
}

In [ ]:
response = requests.post(url,json=customer).json()

In [ ]:
response

In [ ]:
if response['churn']:
    print('sending email to', 'asdx-123d')

# **5.3 Web services: introduction to Flask**(video 3)


## **Links**

* 0.0.0.0 vs localhost: https://stackoverflow.com/a/20778887/861423

In this session we talked about what is a web service and how to create a simple web service.

* What is actually a web service
    * A web service is a method used to communicate between electronic devices.
    * There are some methods in web services we can use it to satisfy our problems. Here below we would list some.
      * GET: GET is a method used to retrieve files, For example whene we are searching for a cat image in google we are actually requesting cat images with GET method.
      * POST: POST is the second common method used in web services. For example in a sign up process, when we are submiting our name, username, passwords, etc we are posting our data to a server that is using the web service. (Note that there is no specification where the data goes)
      * PUT: PUT is same as POST but we are specifying where the data is going to.
      * DELETE: DELETE is a method that is used to request to delete some data from the server.
For more information just google the HTTP methods, You'll find useful information about this.

* To create a simple web service, there are plenty libraries available in every language. Here we would like to introduce Flask library in python.
    * If you haven't installed the library just try installing it with the code pip install Flask
    * To create a simple web service just run the code below:

In [ ]:
from flask import Flask
app = Flask('churn-app') # give an identity to your web service
@app.route('/ping',methods=[GET])
def ping():
    return 'PONG'

if __name__=='__main__':
   app.run('debug=True, host='0.0.0.0', port=9696) # run the code in local machine with the debugging mode true and port 9696

* 
  * With the code above we made a simple web server and created a route named ping that would send pong string.

  * To test it just open your browser and search localhost:9696/ping, You'll see that the 'PONG' string is received. Congrats You've made a simple web server 🥳.

* To use our web server to predict new values we must modify it. See how in the next session.


# **5.4 Serving the churn model with Flask**(video 4)

In this session we talked about implementing the functionality of prediction to our churn web service and how to make it usable in development environment.

* To make the web service predict the churn value for each customer we must modify the code in session 3 with the code we had in previous chapters. Below we can see how the code works in order to predict the churn value.
* In order to predict we need to first load the previous saved model and use a prediction function in a special route.
    * To load the previous saved model we use the code below:


In [ ]:
import pickle

with open('churn-model.bin', 'rb') as f_in:
  dv, model = pickle.load(f_in)

* 
    * As we had earlier to predict a value for a customer we need a function like below:


In [ ]:
def predict_single(customer, dv, model):
  X = dv.transform([customer])  ## apply the one-hot encoding feature to the customer data 
  y_pred = model.predict_proba(X)[:, 1]
  return y_pred[0]

* 
    * Then at last we make the final function used for creating the web service.

In [ ]:
@app.route('/predict', methods=['POST'])  ## in order to send the customer information we need to post its data.
def predict():
customer = request.get_json()  ## web services work best with json frame, So after the user post its data in json format we need to access the body of json.

prediction = predict_single(customer, dv, model)
churn = prediction >= 0.5

result = {
    'churn_probability': float(prediction), ## we need to conver numpy data into python data in flask framework
    'churn': bool(churn),  ## same as the line above, converting the data using bool method
}

return jsonify(result)  ## send back the data in json format to the user

* 
    * The whole code above is available in this link: https://github.com/alexeygrigorev/mlbookcamp-code/blob/master/chapter-05-deployment/churn_serving.py 
    * At last run your code. To see the result can't use a simple request in a web browser. We can run the code below to post new user data and see the response.


In [ ]:
## a new customer informations
customer = {
  'customerid': '8879-zkjof',
  'gender': 'female',
  'seniorcitizen': 0,
  'partner': 'no',
  'dependents': 'no',
  'tenure': 41,
  'phoneservice': 'yes',
  'multiplelines': 'no',
  'internetservice': 'dsl',
  'onlinesecurity': 'yes',
  'onlinebackup': 'no',
  'deviceprotection': 'yes',
  'techsupport': 'yes',
  'streamingtv': 'yes',
  'streamingmovies': 'yes',
  'contract': 'one_year',
  'paperlessbilling': 'yes',
  'paymentmethod': 'bank_transfer_(automatic)',
  'monthlycharges': 79.85,
  'totalcharges': 3320.75
}
import requests ## to use the POST method we use a library named requests
url = 'http://localhost:9696/predict' ## this is the route we made for prediction
response = requests.post(url, json=customer) ## post the customer information in json format
result = response.json() ## get the server response
print(result)

* Until here we saw how we made a simple web server that predicts the churn value for every user. When you run your app you will see a warning that it is not a WGSI server and not suitable for production environmnets. To fix this issue and run this as a production server there are plenty of ways available.
    * One way to create a WSGI server is to use gunicorn. To install it use the command pip install gunicorn, And to run the WGSI server you can simply run it with the command gunicorn --bind 0.0.0.0:9696 churn:app. Note that in churn:app the name churn is the name we set for our the file containing the code app = Flask('churn')(for example: churn.py), You may need to change it to whatever you named your Flask app file.
    * Windows users may not be able to use gunicorn library because windows system do not support some dependecies of the library. So to be able to run this on a windows machine, there is an alternative library waitress and to install it just use the command pip install waitress.
    * to run the waitress wgsi server use the command waitress-serve --listen=0.0.0.0:9696 churn:app.
To test it just you can run the code above and the results is the same.
* So until here you were able to make a production server that predicts the churn value for new customers. In the next session, we can see how to solve library version conflicts in each machine and manage the dependencies for production environments.


# **5.5 Python virtual environment: Pipenv** (video 5)

In this session we're going to make virtual environment for our project. So Let's start this session to get to know what is a virtual environment and how to make it.

* Every time we're running a file from a directory we're using the executive files from a global directory. For example when we install python on our machine the executable files that are able to run our codes will go to somewhere like /home/username/python/bin/ for example the pip command may go to /home/username/python/bin/pip.
* Sometimes the versions of libraries conflict (the project may not run or get into massive errors). For example, we have an old project that uses the sklearn library with version 0.24.1 and now we want to run it using sklearn version 1.0.0. We may get into errors because of the version conflict.
    * To solve the conflict we can make virtual environments. The virtual environment is something that can separate the libraries installed in our system and the libraries with the specified version we want our project to run with. There are a lot of ways to create a virtual environments. One way we are going to use is using a library named pipenv.
    * pipenv is a library that can create a virutal environment. To install this library just use the classic method pip install pipenv.
    * After installing pipenv we must to install the libraries we want for our project in the new virtual environment. It's really easy, Just use the command pipenv instead of pip. pipenv install numpy sklearn==0.24.1 flask. With this command we installed the libraries we want for our project.
    * Note that using the pipenv command we made two files named Pipfile and Pipfile.lock. If we look at this files closely we can see that in Pipfile the libraries we installed are named. If we specified the library name, it's also specified in Pipfile.
    * In Pipfile.lock we can see that each library with it's installed version is named and a hash file is there to reproduce if we move the environment to another machine.
    * If we want to run the project in another machine, we can easily installed the libraries we want with the command pipenv install. This command will look into Pipfile and Pipfile.lock to install the libraries with specified version.
    * After installing the required libraries we can run the project in the virtual environment with the pipenv shell command. This will go to the virtual environment's shell and then any command we execute will use the virtual environment's libraries.
* Installing and using the libraries such as gunicorn is the same as the last session.
* Until here we made a virtual environment for our libraries with a required specified version. To separate this environment more, such as making unicorn be able to run on windows machines we need another way. The other way is using Docker. Docker allows us to separate everything more than before and makes any project able to run on any machine that supports Docker smoothly.
* In the next session, we'll go into detail about how Docker works and how to use it.

# **5.6 Environment management: Docker**(video 6)

## **Installing Docker**
____
To isolate more our project file from our system machine, there is an option named Docker. With Docker you are able to pack all your project is a system that you want and run it in any system machine. For example if you want Ubuntu 20.4 you can have it in a mac or windows machine or other operating systems.
To get started with Docker for the churn prediction project you can follow the instructions below.



## **Ubuntu**

* To install Docker run the command below.



In [ ]:
bash
sudo apt-get install docker.io

* To run docker without sudo, follow this instruction:https://docs.docker.com/engine/install/linux-postinstall/.
    * Once our project was packed in a Docker container, we're able to run our project on any machine.
    * First we have to make a Docker image. In Docker image file there are settings and dependecies we have in our project. To find Docker images that you need you can simply search the Docker website: https://hub.docker.com/search?type=image.
    * Here a Docker file is written we'll explain it below.(There should be no comments in Docker file, So remove the comments if you want to copy it)

In [ ]:
FROM python:3.8.12-slim                                                     # First install the python 3.8, the slim version have less size
RUN pip install pipenv                                                      # Install pipenv library in Docker 
WORKDIR /app                                                                # we have created a directory in Docker named app and we're using it as work directory 
COPY ["Pipfile", "Pipfile.lock", "./"]                                      # Copy the Pip files into our working derectory 
RUN pipenv install --deploy --system                                        # install the pipenv dependecies we had from the project and deploy them 
COPY ["*.py", "churn-model.bin", "./"]                                      # Copy any python files and the model we had to the working directory of Docker 
EXPOSE 9696                                                                 # We need to expose the 9696 port because we're not able to communicate with Docker outside it
ENTRYPOINT ["gunicorn", "--bind", "0.0.0.0:9696", "churn_serving:app"]      # If we run the Docker image, we want our churn app to be running

* If we don't put the last line ENTRYPOINT, we will be in a python shell. Note that in Docker we made put in double quotes, This is because of the spaces. We have to ignore spaces in a command and put the characters in double quotes.(See ENTRYPOINT for example)
    * After creating the Dockerfile and writing the settings we want in it, We need to build it with the command below.

In [ ]:
 docker build -t churn-prediction .

* With -t command We're specifying the name churn-prediction for this Dockerfile.
    * To run it, Simply execute the command below:

In [ ]:
 docker run -it -p 9696:9696 churn-prediction:latest

* Here we use the option -it in order to the Docker run from terminal and shows the result. The -p parameter is used to map the 9696 port of the Docker to 9696 port of our machine.(first 9696 is the port number of our machine and the last one is Docker container port.)
    * At last you've deployed your prediction app inside a Docker continer. Congratulations 🥳


## **Windows**

To install the Docker you can just follow the instruction by Andrew Lock in this link: https://andrewlock.net/installing-docker-desktop-for-windows/

# **5.7 Deployment to the cloud: AWS Elastic Beanstalk (optional)**

## **Links**
____
* Creating an account on AWS:https://mlbookcamp.com/article/aws



## **Notes**
___
As we see how to deploy our apps in AWS Let's find it out how to deploy them in Heroku.



## **Heroku**

Here we will learn how to deploy our apps in Heroku instead of AWS.

* First of all create your web service with flask. (example file: churn_prediction.py
* Then create a file named requirements.txt and assing your dependencies there. Example:


In [ ]:
pickle
numpy
flask
gunicorn

* Create another file named Procfile and add the app you want to be able to run there. Example:


In [ ]:
web: gunicorn churn_serving:app

Note that the churn_serving name in the box above is the name of the main python file we're going to be runned.

* Create your heroku profile, Go to dashboard and the Deploy tab.
* Follow the instruction for Deploy using Heroku Git.
* Great your app is now available from global universe.

I've put my heroku app files in this repository: https://github.com/amindadgar/customer-churn-app



# **5.8 Summary**

## Notes
____
In this chapter we learned this topics:

We learned how to save the model and load it to re-use it without running the previous code.
* How to deploy the model in a web service.
* How to create a virtual environment.
* How to create a container and run our code in any operating system.
* How to implement our code in a public web service and access it from outside a local computer.
In the next chapter, we would learn algorithms such as Decision trees, Random forests, and Gradient boosting as an alternative way of combining decision trees.